In [72]:
import os
import openai
import json
from pathlib import Path
import re

In [73]:
class GPTinterface():
    def __init__(self) -> None:
        filePath=os.path.join(os.getcwd(),"./key")
        with open(filePath,'r') as f:
            keys=f.readlines()
        openai.api_key = keys[0][:-1]
        openai.organization = keys[1]
        self.client = openai.OpenAI(api_key=keys[0][:-1])
        # print(openai.Model.list())
    
    def APIcommunicate(self,messages:list=[],temperature=1,max_tokens=4096,top_p=1,frequency_penalty=0,presence_penalty=0,modelVersion='gpt-4-turbo-preview',response_format='text',):
        if 'vi' in modelVersion.lower():
            modelVersion='gpt-4o'
        elif '3' in modelVersion:
            modelVersion='gpt-3.5-turbo-0125'
        elif '4' in modelVersion:
            modelVersion='gpt-4o'
        response = self.client.chat.completions.create(
        model=modelVersion,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
        top_p=top_p,
        frequency_penalty=frequency_penalty,
        presence_penalty=presence_penalty,
        response_format={ "type":response_format }
        # or "json_object" 
        )
        ret=response.choices[0].message.content
        return ret

In [74]:
llm = GPTinterface()
def merge_tasks(formatted_tasks: str): 
    prompt=[
            {"role": "system",
            "content": """I have a set of primitive subtasks for robots. 
            Help me combine primitive subtasks into a larger task by randomly selecting several primitive subtasks.
            There can be temporal constraints between primitive subtasks. 
            Below is the demonstrating example: 
            The primitive subtasks are 
            1. Place two scoops in the top drawer to the left of the stove 
            2. Put two measuring cups in a drawer on the kitchen island near the sink, 
            3. Gather two ladles and store them in the cabinet to the right of the fridge. 
            One such example is that Place two scoops in the top drawer to the left of the stove at any time. 
            Simultaneously, put two measuring cups in a drawer on the kitchen island near the sink.
            After placing the scoops, gather two ladles and store them in the cabinet to the right of the fridge.
            What I am particularly interested in is that, in one example, some subtasks need to be done sequentially, while some subtasks can be done in any order."""
            },{
            "role": "user",
            "content": """ Below is the new set of primitive subtasks:
            {}
            Give me 5 combined long tasks. Note that you do not need to select all primitive subtasks to generate larger ones.
            Just output the final description of a long task in one line preceding by numbers. No need for explanation. """.format(formatted_tasks)
            }
            ]
    # print(prompt)
    ret = llm.APIcommunicate(prompt,modelVersion='3')
    print(ret)
    return ret

In [76]:
# Define the path to the directory
directory = '/Users/xushengluo/Documents/Code/alfred_multiagent/multiagent_longtasks/'
field_name = 'task_desc'
path = Path(directory)

# Walk through all directories and files in the directory
found_json = False
for dirpath, dirnames, filenames in os.walk(path):
    for file in filenames:
        if file.endswith('.json'):
            # Construct full file path
            file_path = os.path.join(dirpath, file)
            try:
                # Open the JSON file and read data
                with open(file_path, 'r') as json_file:
                    data = json.load(json_file)
                
                primitive_subtasks = [task['task_desc'] for task in data['turk_annotations']['anns']]
                formatted_tasks = ' '.join(f"{index + 1}. {item}" for index, item in enumerate(primitive_subtasks))
                
                # print(primitive_subtasks)
                # print(formatted_tasks)
                merged_tasks = merge_tasks(formatted_tasks)
                items = re.split(r'\d+\.\s+', merged_tasks)
                # The first element of the list will be an empty string due to the split at the start of the text, so remove it
                if items[0] == '':
                    items = items[1:]

                data['long_task_lists'] = [item.strip() for item in items]
                # Write the modified data back to the file
                with open(file_path, 'w') as json_file:
                    json.dump(data, json_file, indent=4)
                new_file_name = 'long_' + file if 'long' not in file else file
                new_filepath = os.path.join(dirpath, new_file_name)
                os.rename(file_path, new_filepath)
                print(f"Updated file: {new_filepath}")
        
            except json.JSONDecodeError as e:
                print(f"Error reading JSON file {file_path}: {e}")
            except FileNotFoundError:
                print(f"File not found: {file_path}")
            except Exception as e:
                print(f"An error occurred with file {file_path}: {e}")
            found_json = True
            break
    if found_json:
        break

1. Put a chilled tomato on the counter left of the stove, then place a cold tomato on the gas cooker.

2. Cool the tomato in the refrigerator, put it by the toaster at any time. Meanwhile, place a cold tomato on the gas cooker.

3. First, cool the tomato in the refrigerator, then put it by the toaster. After that, put a chilled tomato on the counter left of the stove.

4. Simultaneously, place a cold tomato on the gas cooker and cool the tomato in the refrigerator, and put it by the toaster.

5. Put a chilled tomato on the counter left of the stove at any time. Meanwhile, cool the tomato in the refrigerator, then put it by the toaster.
Updated file: /Users/xushengluo/Documents/Code/alfred_multiagent/multiagent_longtasks/json_2.1.0/trial_T20190906_185208_580877/multi_traj_data.json
